# View Molecular Orbitals

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output

import urlparse

import nglview

In [ ]:
# parse URL
url = urlparse.urlsplit(jupyter_notebook_url)
querypart = urlparse.parse_qs(url.query)
pk = int(querypart['pk'][0])
calc = load_node(pk)

In [ ]:
def on_mo_changed(c):
    #while hasattr(viewer, "component_0"):
    if hasattr(viewer, "component_1"):
        viewer.component_1.remove_surface()
        viewer.component_1.remove_surface()
        cid = viewer.component_1.id
        viewer.remove_component(cid)
    
    if inp_mo.value:
        rn = calc.get_retrieved_node()
        fn = rn.get_abs_path(inp_mo.value)
        viewer.add_component(fn, defaultRepresentation=False)
        on_isolevel_change(None)

all_cubes = [fn for fn in calc.get_retrieved_node().get_folder_list() if fn.endswith(".cube")]
inp_mo = ipw.Select(options=all_cubes, height="200px")
inp_mo.observe(on_mo_changed, "value")
display(inp_mo)
if len(all_cubes)==0:
    print("No CUBE files found.")

In [ ]:
def on_isolevel_change(c):
    #http://arose.github.io/ngl/api/typedef/index.html#static-typedef-SurfaceRepresentationParameters
    if not hasattr(viewer, "component_1"):
        return
    viewer.component_1.remove_surface()
    viewer.component_1.remove_surface()
    #viewer.component_1.clear_representations()
    v = sld_isolevel.value
    viewer.component_1.add_surface(isolevel=+v, color="red")
    viewer.component_1.add_surface(isolevel=-v, color="blue")

sld_isolevel = ipw.FloatSlider(value=0.1, min=0, max=3.0, step=0.05, continuous_update=False)
sld_isolevel.observe(on_isolevel_change, "value")
display(ipw.HBox([ipw.Label("Isolevel:"), sld_isolevel]))

In [ ]:
viewer = nglview.NGLWidget()
atoms = calc.inp.structure.get_ase()
viewer.add_component(nglview.ASEStructure(atoms), defaultRepresentation=False)
viewer.add_unitcell(component=0)
viewer.add_ball_and_stick(component=0)
viewer.center_view(component=0)
display(viewer)

# update view
on_mo_changed(None)

In [ ]:
viewer.component_1